# Data preprocessing

## Important functions to be used multiple times

In [7]:
import pandas as pd
import numpy as np
import mysql.connector

# Calculate the z-score for a specified column
def get_z_scores(column_name):
    z_scores = np.abs((df[column_name] - df[column_name].mean()) / df[column_name].std())
    return z_scores

# Remove outliers from the DataFrame based on z-scores
def remove_outliers(z_scores):
    # Define a threshold for outlier detection (e.g., z-score > 3)
    threshold = 3

    # Filter rows where z-score exceeds the threshold
    outliers = df[z_scores > threshold]

    # Remove outliers from the DataFrame
    cleaned_df = df[z_scores <= threshold]
    return cleaned_df

## HS_Cigar_Use

In [9]:
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Read data from the MySQL table into a pandas DataFrame
query = "SELECT * FROM cdc_cigar_use"
df = pd.read_sql(query, conn)

# Close the MySQL connection
conn.close()

# Specify the columns with potential outliers
dirty_columns = [
    'Greater_Risk_Data_Value',
    'Greater_Risk_Low_Confidence_Limit',
    'Greater_Risk_High_Confidence_Limit',
    'Lesser_Risk_Data_Value',
    'Lesser_Risk_Low_Confidence_Limit',
    'Lesser_Risk_High_Confidence_Limit'
]

# Iterate through the columns to remove outliers
for column in dirty_columns:
    z_scores = get_z_scores(column)
    cleaned_df = remove_outliers(z_scores)

# Reconnect
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Truncate the existing table to remove all data
truncate_query = "TRUNCATE TABLE cdc_cigar_use"
cursor.execute(truncate_query)
conn.commit()

# Insert the cleaned data into the MySQL table
insert_query = (
    "INSERT INTO cdc_cigar_use "
    "(YEAR, LocationAbbr, LocationDesc, DataSource, Topic, Subtopic, "
    "ShortQuestionText, Greater_Risk_Question, Description, Data_Value_Symbol, "
    "Data_Value_Type, Greater_Risk_Data_Value, Greater_Risk_Data_Value_Footnote_Symbol, "
    "Greater_Risk_Data_Value_Footnote, Greater_Risk_Low_Confidence_Limit, "
    "Greater_Risk_High_Confidence_Limit, Lesser_Risk_Question, Lesser_Risk_Data_Value, "
    "Lesser_Risk_Data_Value_Footnote_Symbol, Lesser_Risk_Data_Value_Footnote, "
    "Lesser_Risk_Low_Confidence_Limit, Lesser_Risk_High_Confidence_Limit, Sample_Size, "
    "Sex, Race, Grade, SexualIdentity, SexOfSexualContacts, GeoLocation, TopicId, "
    "SubTopicID, QuestionCode, LocationId, StratID1, StratID2, StratID3, StratID4, "
    "StratID5, StratificationType, StratID6) "
    "VALUES "
    "(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, "
    "%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)

for row in cleaned_df.itertuples(index=False):
    cursor.execute(insert_query, row)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Ouliers in data were cleaned and inserted successfully!")

Outliers removed and inserted successfully!


/var/folders/9q/yldjq0js2ng8cysrh56tt9xw0000gn/T/ipykernel_38943/708248302.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [6]:
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()
# Read data from the MySQL table into a pandas DataFrame
query = "SELECT * FROM cms_chronic_conditions"
df = pd.read_sql(query, conn)

# Close the MySQL connection
conn.close()

# Columns to clean and remove outliers
dirty_columns = [
    'Prvlnc',
    'Tot_Mdcr_Stdzd_Pymt_PC',
    'Tot_Mdcr_Pymt_PC',
    'Hosp_Readmsn_Rate',
    'ER_Visits_Per_1000_Benes',
    ]

# Iterate through the columns to remove outliers
for column in dirty_columns:
    z_scores = get_z_scores(column)
    df = remove_outliers(z_scores)

# Reconnect to MySQL for data insertion
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Truncate the existing table to remove all data
truncate_query = "TRUNCATE TABLE cms_chronic_conditions"
cursor.execute(truncate_query)
conn.commit()

# Batch insert the cleaned data into the MySQL table
insert_query = (
    "INSERT INTO cms_chronic_conditions "
    "(Bene_Geo_Lvl, Bene_Geo_Desc, Bene_Geo_Cd, Bene_Age_Lvl, Bene_Demo_Lvl, "
    "Bene_Demo_Desc, Bene_Cond, Prvlnc, Tot_Mdcr_Stdzd_Pymt_PC, Tot_Mdcr_Pymt_PC, "
    "Hosp_Readmsn_Rate, ER_Visits_Per_1000_Benes) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)

# Convert DataFrame to list of tuples for batch insert
data_to_insert = df.values.tolist()
cursor.executemany(insert_query, data_to_insert)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Outliers in data were cleaned and inserted successfully!")

Outliers in data were cleaned and inserted successfully!


/var/folders/9q/yldjq0js2ng8cysrh56tt9xw0000gn/T/ipykernel_40329/3956154861.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## CVD Cleaned

In [8]:
# Connect to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Read data from the MySQL table into a pandas DataFrame
query = "SELECT * FROM cdv_cleaned"
df = pd.read_sql(query, conn)

# Close the MySQL connection
conn.close()

# Columns to clean and remove outliers
dirty_columns = [
    'Height_cm',
    'Weight_kg',
    'BMI',
    'Alcohol_Consumption',
    'Fruit_Consumption',
    'Green_Vegetables_Consumption',
    'FriedPotato_Consumption',
]

# Iterate through the columns to remove outliers
for column in dirty_columns:
    z_scores = get_z_scores(column)
    df = remove_outliers(z_scores)

# Reconnect to MySQL for data insertion
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Truncate the existing table to remove all data
truncate_query = "TRUNCATE TABLE cdv_cleaned"
cursor.execute(truncate_query)
conn.commit()

# Batch insert the cleaned data into the MySQL table
insert_query = (
    "INSERT INTO cdv_cleaned "
    "(General_Health, Checkup, Exercise, Heart_Disease, Skin_Cancer, Other_Cancer, "
    "Depression, Diabetes, Arthritis, Sex, Age_Category, Height_cm, Weight_kg, "
    "BMI, Smoking_History, Alcohol_Consumption, Fruit_Consumption, "
    "Green_Vegetables_Consumption, FriedPotato_Consumption) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)

# Convert DataFrame to list of tuples for batch insert
data_to_insert = df.values.tolist()
cursor.executemany(insert_query, data_to_insert)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Outliers in CVD data were cleaned and inserted successfully!")

/var/folders/9q/yldjq0js2ng8cysrh56tt9xw0000gn/T/ipykernel_40329/2606781736.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Outliers in CVD data were cleaned and inserted successfully!


## Heart 2020 Cleaned

In [10]:
# Connect to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Read data from the MySQL table into a pandas DataFrame
query = "SELECT * FROM heart_2020_cleaned"
df = pd.read_sql(query, conn)

# Close the MySQL connection
conn.close()

# Columns to clean and remove outliers
dirty_columns = [
    'BMI',
    'PhysicalHealth',
    'MentalHealth',
    'SleepTime',
]

# Iterate through the columns to remove outliers
for column in dirty_columns:
    z_scores = get_z_scores(column)
    df = remove_outliers(z_scores)

# Reconnect to MySQL for data insertion
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Truncate the existing table to remove all data
truncate_query = "TRUNCATE TABLE heart_2020_cleaned"
cursor.execute(truncate_query)
conn.commit()

# Batch insert the cleaned data into the MySQL table
insert_query = (
    "INSERT INTO heart_2020_cleaned "
    "(HeartDisease, BMI, Smoking, AlcoholDrinking, Stroke, PhysicalHealth, MentalHealth, "
    "DiffWalking, Sex, AgeCategory, Race, Diabetic, PhysicalActivity, GenHealth, "
    "SleepTime, Asthma, KidneyDisease, SkinCancer) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)

# Convert DataFrame to list of tuples for batch insert
data_to_insert = df.values.tolist()
cursor.executemany(insert_query, data_to_insert)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Outliers in Heart 2020 Cleaned data were cleaned and inserted successfully!")

/var/folders/9q/yldjq0js2ng8cysrh56tt9xw0000gn/T/ipykernel_40329/1397861756.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Outliers in Heart 2020 Cleaned data were cleaned and inserted successfully!


## Heart 2022 with nans

In [12]:
# Connect to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Read data from the MySQL table into a pandas DataFrame
query = "SELECT * FROM heart_2022_with_nans"
df = pd.read_sql(query, conn)

# Close the MySQL connection
conn.close()

# Columns to clean and remove outliers
dirty_columns = [
    'HeightInMeters',
    'WeightInKilograms',
    'BMI',
    'PhysicalHealthDays',
    'MentalHealthDays',
    'SleepHours',
]

# Iterate through the columns to remove outliers
for column in dirty_columns:
    z_scores = get_z_scores(column)
    df = remove_outliers(z_scores)

# Reconnect to MySQL for data insertion
conn = mysql.connector.connect(
    host="localhost",
    user="monty",
    password="sushiSQL",
    database="HealthCareAnalytics"
)
cursor = conn.cursor()

# Truncate the existing table to remove all data
truncate_query = "TRUNCATE TABLE heart_2022_with_nans"
cursor.execute(truncate_query)
conn.commit()

# Batch insert the cleaned data into the MySQL table
insert_query = (
    "INSERT INTO heart_2022_with_nans "
    "(State, Sex, GeneralHealth, PhysicalHealthDays, MentalHealthDays, LastCheckupTime, "
    "PhysicalActivities, SleepHours, RemovedTeeth, HadHeartAttack, HadAngina, HadStroke, "
    "HadAsthma, HadSkinCancer, HadCOPD, HadDepressiveDisorder, HadKidneyDisease, "
    "HadArthritis, HadDiabetes, DeafOrHardOfHearing, BlindOrVisionDifficulty, "
    "DifficultyConcentrating, DifficultyWalking, DifficultyDressingBathing, "
    "DifficultyErrands, SmokerStatus, ECigaretteUsage, ChestScan, RaceEthnicityCategory, "
    "AgeCategory, HeightInMeters, WeightInKilograms, BMI, AlcoholDrinkers, HIVTesting, "
    "FluVaxLast12, PneumoVaxEver, TetanusLast10Tdap, HighRiskLastYear, CovidPos) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, "
    "%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)

# Convert DataFrame to list of tuples for batch insert
data_to_insert = df.values.tolist()
cursor.executemany(insert_query, data_to_insert)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Outliers in Heart 2022 data with NaN values were cleaned and inserted successfully!")

/var/folders/9q/yldjq0js2ng8cysrh56tt9xw0000gn/T/ipykernel_40329/3445979494.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


OperationalError: 1153 (08S01): Got a packet bigger than 'max_allowed_packet' bytes